Nem fut le a trainelés. A forward ciklus nem kapja meg jól az edge featuret amit passzolni kéne, vagy megkapja de nem csinál vele semmit.

out csak a disease feature-öket tartalmazza, edge és genet nem!!!!!!

Negatív éleket is használunk a traineléshez, azt nem tudom hogy jól-e.

Edge feature-öket is kell unsqueezelni?

HeteroData:

-gene nodes -> DPI, DSI

-disease nodes -> fatality rate (random dummy feature)

-edges -> score

In [1]:
# prompt: read data from disgenet_gda_summary_filtered.csv

import pandas as pd

# Assuming the file is in the current working directory.
# If not, provide the correct path.
try:
  df = pd.read_csv('disgenet_gda_summary_filtered.csv')
  print(df.head()) # Print the first few rows to verify
except FileNotFoundError:
  print("Error: File 'disgenet_gda_summary_filtered.csv' not found.")
except pd.errors.ParserError:
  print("Error: Could not parse the CSV file. Please check its format.")
except Exception as e:
  print(f"An unexpected error occurred: {e}")


   geneNcbiID  geneDSI  geneDPI                           diseaseName  score
0        3630    0.254    0.957                       Type 2 Diabetes    1.0
1        2099    0.298    0.957                         Breast Cancer    1.0
2        2099    0.298    0.957                         Breast Cancer    1.0
3        2064    0.298    0.957                         Breast Cancer    1.0
4        3630    0.254    0.957  Diabetes Mellitus, Insulin-Dependent    1.0


In [2]:
# prompt: select the unique disease Names from df

unique_disease_names = df['diseaseName'].unique()
#unique_disease_names

In [3]:
replacements = {
    "Alzheimer's dementia senile and presenile forms": 'Alzheimer Disease',
    "familial alzheimer's disease": 'Alzheimer Disease',
    'AMYOTROPHIC LATERAL SCLEROSIS-PARKINSONISM/DEMENTIA COMPLEX 1': 'Parkinson Disease',
    "Parkinson's Disease": 'Parkinson Disease',
    "Primary Parkinsonism or Parkinson's disease": 'Parkinson Disease',
    "Idiopathic Parkinsonism or Parkinson's disease": 'Parkinson Disease',
    "Huntington's Disease": 'Huntington Disease',
    'HUNTINGTON DIS JUVENILE ONSET': 'Huntington Disease',
    "Dementia in Huntington's disease": 'Huntington Disease',
    'Dystrophies, Muscular':'Duchenne Muscular Dystrophy',
    'MUSCULAR DYSTROPHY, BECKER TYPE':'Duchenne Muscular Dystrophy',
    'Anemia, Diamond Blackfan':'Sickle Cell Anemia',
    'Anemia, Congenital Hypoplastic':'Sickle Cell Anemia',
    'CARCINOMA OF BREAST':'Breast Cancer',
    'Malignant neoplasm of lower lobe, bronchus or lung':'Lung Cancer',
    'Malignant neoplasm of other parts of bronchus or lung':'Lung Cancer',
    'Malignant neoplasm of middle lobe, bronchus or lung':'Lung Cancer',
    'Malignant neoplasm of upper lobe, bronchus or lung':'Lung Cancer',
    'CARCINOMA OF PROSTATE':'Prostate Cancer',
    'SYSTEMIC LUPUS ERYTHEMATOSIS':'Systemic lupus erythematosus',
    'SYSTEMIC LUPUS ERYTHEMATOSUS 16':'Systemic lupus erythematosus',
    'Bipolar I disorder':'Bipolar Disorder',
    'Mixed bipolar disorder, NOS':'Bipolar Disorder',
    'Bipolar II disorder':'Bipolar Disorder',
    'diabetes (mellitus) due to autoimmune process':'Type 1 Diabetes',
    'diabetes (mellitus) due to immune mediated pancreatic islet beta-cell destruction':'Type 1 Diabetes',
    'Diabetes Mellitus, Insulin-Dependent':'Type 1 Diabetes',
    'ketosis prone diabetes':'Type 2 Diabetes',
    'idiopathic diabetes (mellitus)':'Type 2 Diabetes',
    'Insulin-resistant diabetes mellitus':'Type 2 Diabetes',
    'NIDDM1':'Type 2 Diabetes',
    'CRC': 'Colorectal Cancer',
    'Malignant neoplasm of colon': 'Colorectal Cancer',
    'Malignant neoplasm of large intestine': 'Colorectal Cancer',
    'Congenital hypothyroidism':'Hypothyroidism',
    'secondary hypothyroidism (diagnosis)':'Hypothyroidism',
    'Primary hypothyroidism':'Hypothyroidism',
    'central hypothyroidism':'Hypothyroidism'
}

# Use the replace method with the dictionary
df['diseaseName'] = df['diseaseName'].replace(replacements)

unique_disease_names = df['diseaseName'].unique()
unique_disease_names

array(['Type 2 Diabetes', 'Breast Cancer', 'Type 1 Diabetes',
       'Alzheimer Disease', 'Parkinson Disease', 'Cystic Fibrosis',
       'Huntington Disease', 'Colorectal Cancer',
       'Amyotrophic Lateral Sclerosis (ALS)',
       'Duchenne Muscular Dystrophy', 'Rheumatoid arthritis',
       'Frontotemporal dementia', 'Epilepsies', 'Schizophrenias',
       'Cardiovascular Diseases', 'Bipolar Disorder',
       'Systemic lupus erythematosus', 'Sickle Cell Anemia',
       'Hypothyroidism', 'Lung Cancer', 'Prostate Cancer', 'PPND',
       'Manic Disorders'], dtype=object)

In [4]:
# prompt: Eliminate duplicate rows in df

df = df.drop_duplicates()
df

,geneNcbiID,geneDSI,geneDPI,diseaseName,score
0,3630,0.254,0.957,Type 2 Diabetes,1.0
1,2099,0.298,0.957,Breast Cancer,1.0
3,2064,0.298,0.957,Breast Cancer,1.0
4,3630,0.254,0.957,Type 1 Diabetes,1.0
5,4137,0.351,0.957,Alzheimer Disease,1.0
...,...,...,...,...,...
6408,23233,0.664,0.478,Schizophrenias,0.4
6409,389207,0.751,0.261,Schizophrenias,0.4
6410,10667,0.572,0.826,Schizophrenias,0.4
6411,200424,0.533,0.913,Schizophrenias,0.4


In [5]:
# prompt: enumerate all disease names in a new variable diseases also genes

diseases = list(unique_disease_names)
genes = df['geneNcbiID'].unique()

In [6]:
!pip install torch-geometric
from torch_geometric.data import HeteroData
import torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.4 MB/s eta 0:00:00


In [7]:
gene_to_index = {gene_id: idx for idx, gene_id in enumerate(df['geneNcbiID'].unique())}
disease_to_index = {disease: idx for idx, disease in enumerate(df['diseaseName'].unique())}

In [8]:
# Convert genes and diseases to their respective indices
df['gene_index'] = df['geneNcbiID'].map(gene_to_index)
df['disease_index'] = df['diseaseName'].map(disease_to_index)

# Create edge index tensor
edge_index = torch.tensor(df[['gene_index', 'disease_index']].values).t().contiguous()

In [112]:
# Initialize the HeteroData object
data = HeteroData()

# Add node features for 'gene' nodes
gene_features = df.groupby('gene_index')[['geneDSI','geneDPI']].first().sort_index().values
data['gene'].x = torch.tensor(gene_features, dtype=torch.float)


# Add disease nodes with calculated lethality features
disease_features = ([0.1000, 0.3000, 0.1500, 0.7000, 0.3000, 0.8000, 0.8000, 0.4000, 0.9000,
        0.8500, 0.1000, 0.7500, 0.0500, 0.2000, 0.7000, 0.1500, 0.2500, 0.4000,
        0.0500, 0.8500, 0.3000, 0.0100, 0.0500])
data['disease'].x = torch.tensor(disease_features, dtype=torch.float).unsqueeze(1)

# Add edges and edge features
data['gene', 'associated_with', 'disease'].edge_index = edge_index
data['gene', 'associated_with', 'disease'].edge_attr = torch.tensor(df['score'].values, dtype=torch.float)#.unsqueeze(1)

In [94]:
data

HeteroData(
  gene={ x=[3608, 2] },
  disease={ x=[23, 1] },
  (gene, associated_with, disease)={
    edge_index=[2, 5958],
    edge_attr=[5958],
  }
)

In [155]:
# prompt: Create another GNN using convolutional layer GraphConv that is capable of passing edge features for this bipartite graph

from torch_geometric.nn import HeteroConv, SAGEConv, GraphConv

# Define the GNN model
class HeteroGNN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()

        # Define a dictionary of convolutional layers for each edge type
        self.convs = HeteroConv({
            ('gene', 'associated_with', 'disease'): GraphConv((-1, -1), hidden_channels, aggr='mean'),
        }, aggr='sum')
        self.lin = torch.nn.Linear(hidden_channels, 1) # Output layer

    def forward(self, x_dict, edge_index_dict, edge_weight_dict):
        # Perform message passing
        x_dict = self.convs(x_dict, edge_index_dict, edge_weight_dict=edge_weight_dict)

        # Get the output for disease nodes
        x_disease = x_dict['disease']

        # Apply a linear transformation to the output
        return self.lin(x_disease)


In [113]:
import random

def create_train_test_split(data, split_ratio=0.8):
    # Get the edge indices for the 'gene' -> 'disease' relationship
    edge_index = data['gene', 'associated_with', 'disease'].edge_index

    # Randomly shuffle the edge indices
    num_edges = edge_index.shape[1]
    indices = list(range(num_edges))
    random.shuffle(indices)

    # Split into train and test sets
    train_size = int(split_ratio * num_edges)
    train_indices = indices[:train_size]
    test_indices = indices[train_size:]

    # Create masks for train and test edges
    train_mask = torch.zeros(num_edges, dtype=torch.bool)
    test_mask = torch.zeros(num_edges, dtype=torch.bool)

    # Set the train and test masks
    train_mask[train_indices] = 1
    test_mask[test_indices] = 1

    # Assign the masks to the data object
    data['gene', 'associated_with', 'disease'].train_mask = train_mask
    data['gene', 'associated_with', 'disease'].test_mask = test_mask

    return data

# Example usage:
data = create_train_test_split(data)

In [151]:
print(data['gene', 'associated_with', 'disease'].edge_attr)

tensor([1.0000, 1.0000, 1.0000,  ..., 0.4000, 0.4000, 0.4000])


In [12]:
# Check the first few edges
#print("First few gene-disease edges:\n", data['gene', 'associated_with', 'disease'].train_mask[:20])

# Check specific gene node features
#print("First gene node features:\n", data['gene'].x[:5])

In [165]:
import torch.optim as optim
from torch_geometric.utils import negative_sampling

# Example usage
model = HeteroGNN(hidden_channels=64)


optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()  # Binary cross-entropy for link prediction

def train(data):
    model.train()
    optimizer.zero_grad()
    # Assuming you have your data in the 'data' object
    out = model(data.x_dict, data.edge_index_dict, edge_weight_dict={
        ('gene', 'associated_with', 'disease'): data['gene', 'associated_with', 'disease'].edge_attr
    })

    # Select positive edges based on the train mask
    # Extract edge_index and train_mask
    edge_index = data['gene', 'associated_with', 'disease'].edge_index
    train_mask = data['gene', 'associated_with', 'disease'].train_mask

    # Use train_mask to select only the edges where the mask is True
    pos_edge_index = edge_index[:, train_mask.bool()]

    print("pos_edge_index shape:", pos_edge_index.shape)
    print("out", out)

    pos_pred = (out['gene'][pos_edge_index[0]] * out['disease'][pos_edge_index[1]]).sum(dim=-1)

    # Negative edges (sampled)
    neg_edge_index = negative_sampling(
        edge_index=pos_edge_index,
        num_nodes=(data['gene'].num_nodes, data['disease'].num_nodes),
        num_neg_samples=pos_edge_index.size(1)
    )
    neg_pred = (out['gene'][neg_edge_index[0]] * out['disease'][neg_edge_index[1]]).sum(dim=-1)

    # Compute loss
    loss = criterion(pos_pred, torch.ones_like(pos_pred))
    loss += criterion(neg_pred, torch.zeros_like(neg_pred))
    loss.backward()
    optimizer.step()
    return loss.item()

/usr/local/lib/python3.10/dist-packages/torch_geometric/nn/conv/hetero_conv.py:76: UserWarning: There exist node types ({'gene'}) whose representations do not get updated during message passing as they do not occur as destination type in any edge type. This may lead to unexpected behavior.
  warnings.warn(


In [110]:
data

HeteroData(
  gene={ x=[3608, 2] },
  disease={ x=[23] },
  (gene, associated_with, disease)={
    edge_index=[2, 5958],
    edge_attr=[5958, 1],
    train_mask=[5958],
    test_mask=[5958],
  }
)

In [160]:
model.convs(data.x_dict, data.edge_index_dict)

{'disease': tensor([[ 0.2460, -0.7931, -0.4426,  ..., -0.3759, -0.5072, -0.2086],
         [ 0.0135, -0.8206, -0.5589,  ..., -0.1316, -0.4454, -0.1782],
         [ 0.2218, -0.8537, -0.5201,  ..., -0.3844, -0.5561, -0.2260],
         ...,
         [ 0.0565, -0.8887, -0.6202,  ..., -0.2197, -0.5270, -0.2100],
         [ 0.3298, -0.7501, -0.3695,  ..., -0.4558, -0.4974, -0.2024],
         [ 0.2806, -0.7488, -0.3797,  ..., -0.3884, -0.4778, -0.1988]],
        grad_fn=<AddBackward0>)}

In [125]:
from sklearn.metrics import roc_auc_score

@torch.no_grad()
def test(data):
    model.eval()
    out = model(data)

    # Positive and negative edges as in train function
    # Extract edge_index and train_mask
    edge_index = data['gene', 'associated_with', 'disease'].edge_index
    train_mask = data['gene', 'associated_with', 'disease'].test_mask

    # Use train_mask to select only the edges where the mask is True
    pos_edge_index = edge_index[:, train_mask.bool()]
    pos_pred = (out['gene'][pos_edge_index[0]] * out['disease'][pos_edge_index[1]]).sum(dim=-1)

    neg_edge_index = negative_sampling(
        edge_index=pos_edge_index, num_nodes=(data['gene'].num_nodes, data['disease'].num_nodes),
        num_neg_samples=pos_edge_index.size(1)
    )
    neg_pred = (out['gene'][neg_edge_index[0]] * out['disease'][neg_edge_index[1]]).sum(dim=-1)

    # Compute AUC
    y_true = torch.cat([torch.ones(pos_pred.size(0)), torch.zeros(neg_pred.size(0))], dim=0)
    y_score = torch.cat([pos_pred, neg_pred], dim=0)
    auc = roc_auc_score(y_true.cpu(), y_score.cpu())
    return auc

In [166]:
for epoch in range(1, 101):
    loss = train(data)
    if epoch % 10 == 0:
        auc = test(data)
        print(f"Epoch: {epoch:03d}, Loss: {loss:.4f}, AUC: {auc:.4f}")

pos_edge_index shape: torch.Size([2, 4766])
out tensor([[ 0.1965],
        [ 0.1482],
        [ 0.1877],
        [ 0.0464],
        [ 0.1473],
        [ 0.0250],
        [ 0.0232],
        [ 0.1259],
        [-0.0072],
        [ 0.0058],
        [ 0.2011],
        [ 0.0340],
        [ 0.2105],
        [ 0.1720],
        [ 0.0526],
        [ 0.1880],
        [ 0.1601],
        [ 0.1233],
        [ 0.2120],
        [ 0.0183],
        [ 0.1483],
        [ 0.2295],
        [ 0.2210]], grad_fn=<AddmmBackward0>)


IndexError: too many indices for tensor of dimension 2